<a href="https://colab.research.google.com/github/aalvarez359/3dunet-vs-medam-brain-tumor/blob/main/inference/msd/3d_unet_msd_inf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run 1, 2, 3 for U-Net MSD 4 modality Inference

Run 1, 2, 4 for U-Net MSD FLAIR-only Inference

#1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#2

In [ ]:
!pip install nibabel segmentation-models-pytorch-3d torch torchvision torchaudio tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.8/626.8 kB 53.6 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=3aebd3e1659a7d41a32ef4b8cc7e6fe6169104428f62ab4a16723dcf99b166db
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=e188089544640a8636f4537cf1bcbb18ce875d8bcc5862b8d328889b132949a3
  Stored in directory: /root/.cache/pip/wheels/4c/01/56/40a48f7

#3

In [ ]:
# msd_unet4mod_inference.py

import os, json
from pathlib import Path

import numpy as np
import nibabel as nib
from tqdm import tqdm

import torch
import torch.nn.functional as F
import segmentation_models_pytorch_3d as smp3d

# -------------------------
# Paths / config
# -------------------------
JSON_PATH = "/content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/dataset.json"
SAVE_DIR  = "/content/drive/MyDrive/BrainTumor_Checkpoints"   # same as training
SPLIT_JSON_PATH = os.path.join(SAVE_DIR, "dataset_split.json")

# where to save predicted segmentations and CSV
OUTPUT_ROOT = "/content/drive/MyDrive/MSD_UNet4mod_test_preds"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

IN_CHANNELS = 4
N_CLASSES   = 4
PATCH_SIZE  = (128, 224, 224)
OVERLAP     = 0.75

# which checkpoint to use
CKPT_PATH = os.path.join(SAVE_DIR, "last.pt")
#CKPT_PATH = os.path.join(SAVE_DIR, "best_epoch65_dice0.7330.pt")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# -------------------------
# NIfTI I/O + preprocessing
# -------------------------
def percentile_clip(arr, lo=0.5, hi=99.5):
    a = arr.astype(np.float32)
    l, h = np.percentile(a, [lo, hi])
    a = np.clip(a, l, h)
    return a

def zscore_per_channel(x, eps=1e-8):
    # x: (C, D, H, W)
    x = x.astype(np.float32)
    for c in range(x.shape[0]):
        v = x[c]
        mask = (v != 0)
        m = v[mask].mean() if mask.any() else v.mean()
        s = v[mask].std()  if mask.any() else v.std()
        s = max(float(s), eps)
        x[c] = (v - m) / s
    return x

def load_nifti_image_chwd(path_img):
    """
    MSD/Task01 format: (H, W, D, 4) -> (C=4, D, H, W)
    """
    img = nib.load(str(path_img))
    arr = img.get_fdata(dtype=np.float32)
    assert arr.ndim == 4 and arr.shape[3] == IN_CHANNELS, f"{path_img} shape {arr.shape} not HWD4"
    chans = [percentile_clip(arr[..., c], 0.5, 99.5) for c in range(arr.shape[3])]
    vol = np.stack(chans, axis=3)                 # H W D C
    vol = np.transpose(vol, (3, 2, 0, 1))         # C D H W
    vol = zscore_per_channel(vol)
    return vol

def load_nifti_label_dhw(path_lab):
    lab = nib.load(str(path_lab)).get_fdata(dtype=np.float32)  # H W D
    lab = np.rint(lab).astype(np.int64)
    lab = np.transpose(lab, (2, 0, 1))  # D H W
    return lab

# -------------------------
# Sliding-window inference
# -------------------------
@torch.no_grad()
def sliding_window_inference(volume_cdhw, model, window, overlap, device=DEVICE):
    model.eval()
    C, D, H, W = volume_cdhw.shape
    wd, wh, ww = window
    sd = max(1, int(wd * (1 - overlap)))
    sh = max(1, int(wh * (1 - overlap)))
    sw = max(1, int(ww * (1 - overlap)))

    out_prob = torch.zeros((N_CLASSES, D, H, W), dtype=torch.float32, device=device)
    out_norm = torch.zeros((1, D, H, W), dtype=torch.float32, device=device)

    for z in range(0, max(D - wd + 1, 1), sd):
        z0 = min(z, D - wd)
        for y in range(0, max(H - wh + 1, 1), sh):
            y0 = min(y, H - wh)
            for x in range(0, max(W - ww + 1, 1), sw):
                x0 = min(x, W - ww)
                patch = volume_cdhw[:, z0:z0+wd, y0:y0+wh, x0:x0+ww]
                pt = torch.from_numpy(patch).unsqueeze(0).to(device)  # 1,C,D,H,W
                logits = model(pt)                                    # 1,C,d,h,w
                probs = F.softmax(logits, dim=1)[0]                   # C,d,h,w
                out_prob[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += probs
                out_norm[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += 1.0

    out_prob /= (out_norm + 1e-8)
    pred = torch.argmax(out_prob, dim=0)  # D H W
    return pred.cpu().numpy()  # return as numpy, int64

# -------------------------
# Metrics
# -------------------------
def dice_per_class(pred_dhw, target_dhw, n_classes=N_CLASSES):
    pred = pred_dhw.astype(np.int64)
    target = target_dhw.astype(np.int64)
    out = []
    for c in range(n_classes):
        p = (pred == c)
        t = (target == c)
        inter = (p & t).sum()
        denom = p.sum() + t.sum()
        out.append((2.0 * inter / denom) if denom > 0 else 1.0)
    return out  # [dice_bg, dice_edema, dice_netc, dice_et]

def dice_whole_tumor(pred_dhw, target_dhw):
    """
    Whole tumor = union of labels 1,2,3.
    """
    p = (pred_dhw > 0)
    t = (target_dhw > 0)
    inter = (p & t).sum()
    denom = p.sum() + t.sum()
    if denom == 0:
        return 1.0
    return 2.0 * inter / denom

# -------------------------
# Model builder + ckpt load
# -------------------------
def build_model(device):
    model = smp3d.Unet(
        encoder_name="efficientnet-b7",
        encoder_weights=None,
        in_channels=IN_CHANNELS,
        classes=N_CLASSES,
        decoder_channels=(192, 128, 64, 32, 16),
    )
    return model.to(device)

def load_model_for_inference(ckpt_path, device):
    model = build_model(device)
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()
    return model

# -------------------------
# NIfTI saving
# -------------------------
def save_pred_as_nifti(pred_dhw, reference_img_path, out_path):
    # pred_dhw (D,H,W) -> (H,W,D)
    pred_hwd = np.transpose(pred_dhw.astype(np.uint8), (1, 2, 0))
    ref_nii = nib.load(reference_img_path)
    pred_img = nib.Nifti1Image(pred_hwd, affine=ref_nii.affine, header=ref_nii.header)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    nib.save(pred_img, out_path)

# -------------------------
# Load test split
# -------------------------
def load_test_items():
    assert os.path.exists(SPLIT_JSON_PATH), f"Split file not found: {SPLIT_JSON_PATH}"
    with open(SPLIT_JSON_PATH, "r") as f:
        split = json.load(f)
    test_items = split["test"]
    print(f"Loaded test split: {len(test_items)} cases")
    return test_items

# -------------------------
# Main inference loop
# -------------------------
def run_inference():
    model = load_model_for_inference(CKPT_PATH, DEVICE)
    test_items = load_test_items()

    all_results = []
    for it in tqdm(test_items, desc="U-Net 4-mod MSD test inference"):
        img_path = it["image"]
        lab_path = it["label"]  # MSD has labels even for our held-out split

        case_id = os.path.basename(img_path).replace(".nii.gz", "")
        print(f"\nCase: {case_id}")
        print(" Image:", img_path)
        print(" Label:", lab_path)

        vol_cdhw = load_nifti_image_chwd(img_path)
        lab_dhw  = load_nifti_label_dhw(lab_path)

        pred_dhw = sliding_window_inference(vol_cdhw, model, PATCH_SIZE, OVERLAP, DEVICE)

        dices = dice_per_class(pred_dhw, lab_dhw)
        dice_bg, dice_edema, dice_netc, dice_et = dices
        dice_wt = dice_whole_tumor(pred_dhw, lab_dhw)
        mean_tumor = float(np.mean(dices[1:]))

        print(f"  Dice: ED={dice_edema:.3f}  NETC={dice_netc:.3f}  ET={dice_et:.3f}  WT={dice_wt:.3f}  mean(1–3)={mean_tumor:.3f}")

        # save prediction as NIfTI
        out_nii_path = os.path.join(OUTPUT_ROOT, f"{case_id}_UNet4mod_pred.nii.gz")
        save_pred_as_nifti(pred_dhw, img_path, out_nii_path)

        all_results.append({
            "case_id": case_id,
            "image_path": img_path,
            "label_path": lab_path,
            "pred_mask_path": out_nii_path,
            "dice_bg": float(dice_bg),
            "dice_edema": float(dice_edema),
            "dice_netc": float(dice_netc),
            "dice_et": float(dice_et),
            "dice_whole_tumor": float(dice_wt),
            "dice_mean_tumor_classes": float(mean_tumor),
        })

    # save CSV
    if all_results:
        import pandas as pd
        df = pd.DataFrame(all_results)
        csv_path = os.path.join(OUTPUT_ROOT, "UNet4mod_MSD_test_results.csv")
        df.to_csv(csv_path, index=False)
        print("\nSaved CSV:", csv_path)

    print("\n✅ Done U-Net 4-mod MSD test inference.")

if __name__ == "__main__":
    run_inference()


Using device: cuda
Loaded test split: 97 cases


U-Net 4-mod MSD test inference:   0%|          | 0/97 [00:00<?, ?it/s]


Case: BRATS_137
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_137.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_137.nii.gz


U-Net 4-mod MSD test inference:   1%|          | 1/97 [00:07<12:16,  7.67s/it]

  Dice: ED=0.799  NETC=0.591  ET=0.910  WT=0.942  mean(1–3)=0.767

Case: BRATS_441
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_441.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_441.nii.gz


...

U-Net 4-mod MSD test inference:  99%|█████████▉| 96/97 [09:15<00:05,  5.82s/it]

  Dice: ED=0.473  NETC=0.320  ET=0.841  WT=0.888  mean(1–3)=0.544

Case: BRATS_455
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_455.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_455.nii.gz


U-Net 4-mod MSD test inference: 100%|██████████| 97/97 [09:21<00:00,  5.79s/it]

  Dice: ED=0.915  NETC=0.458  ET=0.871  WT=0.955  mean(1–3)=0.748



Saved CSV: /content/drive/MyDrive/MSD_UNet4mod_test_preds/UNet4mod_MSD_test_results.csv

✅ Done U-Net 4-mod MSD test inference.


#4

In [ ]:
# msd_unet_flair_inference.py

import os, json
from pathlib import Path

import numpy as np
import nibabel as nib
from tqdm import tqdm

import torch
import torch.nn.functional as F
import segmentation_models_pytorch_3d as smp3d

# -------------------------
# Paths / config
# -------------------------
JSON_PATH = "/content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/dataset.json"

# same split as 4-mod run:
SPLIT_JSON_PATH = "/content/drive/MyDrive/BrainTumor_Checkpoints/dataset_split.json"

# FLAIR-only checkpoints directory
SAVE_DIR  = "/content/drive/MyDrive/BrainTumor_Checkpoints_FLAIR"

# where to save predicted segmentations and CSV
OUTPUT_ROOT = "/content/drive/MyDrive/MSD_UNetFLAIR_test_preds"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

IN_CHANNELS = 1
N_CLASSES   = 4
PATCH_SIZE  = (128, 224, 224)
OVERLAP     = 0.75

CKPT_PATH = os.path.join(SAVE_DIR, "last.pt")
#CKPT_PATH = os.path.join(SAVE_DIR, "best_epoch71_dice0.5087.pt")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# -------------------------
# NIfTI I/O + preprocessing (FLAIR-only)
# -------------------------
def percentile_clip(arr, lo=0.5, hi=99.5):
    a = arr.astype(np.float32)
    l, h = np.percentile(a, [lo, hi])
    a = np.clip(a, l, h)
    return a

def zscore_per_channel(x, eps=1e-8):
    # x: (C, D, H, W)
    x = x.astype(np.float32)
    for c in range(x.shape[0]):
        v = x[c]
        mask = (v != 0)
        m = v[mask].mean() if mask.any() else v.mean()
        s = v[mask].std()  if mask.any() else v.std()
        s = max(float(s), eps)
        x[c] = (v - m) / s
    return x

def load_nifti_image_chwd(path_img):
    """
    FLAIR-only loader for MSD/Task01:

    Original: (H, W, D, 4)
      - take channel 0 (FLAIR)
      - -> (H, W, D)
      - -> (C=1, D, H, W)
    """
    img = nib.load(str(path_img))
    arr = img.get_fdata(dtype=np.float32)
    assert arr.ndim == 4 and arr.shape[3] >= 1, f"{path_img} shape {arr.shape} not HWD4"

    flair = percentile_clip(arr[..., 0], 0.5, 99.5)  # FLAIR channel
    vol = flair[..., None]                           # (H, W, D, 1)
    vol = np.transpose(vol, (3, 2, 0, 1))           # (C=1, D, H, W)
    vol = zscore_per_channel(vol)
    return vol

def load_nifti_label_dhw(path_lab):
    lab = nib.load(str(path_lab)).get_fdata(dtype=np.float32)  # H W D
    lab = np.rint(lab).astype(np.int64)
    lab = np.transpose(lab, (2, 0, 1))  # D H W
    return lab

# -------------------------
# Sliding-window inference
# -------------------------
@torch.no_grad()
def sliding_window_inference(volume_cdhw, model, window, overlap, device=DEVICE):
    model.eval()
    C, D, H, W = volume_cdhw.shape
    wd, wh, ww = window
    sd = max(1, int(wd * (1 - overlap)))
    sh = max(1, int(wh * (1 - overlap)))
    sw = max(1, int(ww * (1 - overlap)))

    out_prob = torch.zeros((N_CLASSES, D, H, W), dtype=torch.float32, device=device)
    out_norm = torch.zeros((1, D, H, W), dtype=torch.float32, device=device)

    for z in range(0, max(D - wd + 1, 1), sd):
        z0 = min(z, D - wd)
        for y in range(0, max(H - wh + 1, 1), sh):
            y0 = min(y, H - wh)
            for x in range(0, max(W - ww + 1, 1), sw):
                x0 = min(x, W - ww)
                patch = volume_cdhw[:, z0:z0+wd, y0:y0+wh, x0:x0+ww]
                pt = torch.from_numpy(patch).unsqueeze(0).to(device)  # 1,C,D,H,W
                logits = model(pt)
                probs = F.softmax(logits, dim=1)[0]
                out_prob[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += probs
                out_norm[:, z0:z0+wd, y0:y0+wh, x0:x0+ww] += 1.0

    out_prob /= (out_norm + 1e-8)
    pred = torch.argmax(out_prob, dim=0)  # D H W
    return pred.cpu().numpy()

# -------------------------
# Metrics
# -------------------------
def dice_per_class(pred_dhw, target_dhw, n_classes=N_CLASSES):
    pred = pred_dhw.astype(np.int64)
    target = target_dhw.astype(np.int64)
    out = []
    for c in range(n_classes):
        p = (pred == c)
        t = (target == c)
        inter = (p & t).sum()
        denom = p.sum() + t.sum()
        out.append((2.0 * inter / denom) if denom > 0 else 1.0)
    return out

def dice_whole_tumor(pred_dhw, target_dhw):
    p = (pred_dhw > 0)
    t = (target_dhw > 0)
    inter = (p & t).sum()
    denom = p.sum() + t.sum()
    if denom == 0:
        return 1.0
    return 2.0 * inter / denom

# -------------------------
# Model + ckpt load
# -------------------------
def build_model(device):
    model = smp3d.Unet(
        encoder_name="efficientnet-b7",
        encoder_weights=None,
        in_channels=IN_CHANNELS,
        classes=N_CLASSES,
        decoder_channels=(192, 128, 64, 32, 16),
    )
    return model.to(device)

def load_model_for_inference(ckpt_path, device):
    model = build_model(device)
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()
    return model

# -------------------------
# NIfTI saving
# -------------------------
def save_pred_as_nifti(pred_dhw, reference_img_path, out_path):
    pred_hwd = np.transpose(pred_dhw.astype(np.uint8), (1, 2, 0))
    ref_nii = nib.load(reference_img_path)
    pred_img = nib.Nifti1Image(pred_hwd, affine=ref_nii.affine, header=ref_nii.header)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    nib.save(pred_img, out_path)

# -------------------------
# Load test split (reused)
# -------------------------
def load_test_items():
    assert os.path.exists(SPLIT_JSON_PATH), f"Split file not found: {SPLIT_JSON_PATH}"
    with open(SPLIT_JSON_PATH, "r") as f:
        split = json.load(f)
    test_items = split["test"]
    print(f"Loaded test split: {len(test_items)} cases")
    return test_items

# -------------------------
# Main inference loop
# -------------------------
def run_inference():
    model = load_model_for_inference(CKPT_PATH, DEVICE)
    test_items = load_test_items()

    all_results = []
    for it in tqdm(test_items, desc="U-Net FLAIR-only MSD test inference"):
        img_path = it["image"]
        lab_path = it["label"]

        case_id = os.path.basename(img_path).replace(".nii.gz", "")
        print(f"\nCase: {case_id}")
        print(" Image:", img_path)
        print(" Label:", lab_path)

        vol_cdhw = load_nifti_image_chwd(img_path)
        lab_dhw  = load_nifti_label_dhw(lab_path)

        pred_dhw = sliding_window_inference(vol_cdhw, model, PATCH_SIZE, OVERLAP, DEVICE)

        dices = dice_per_class(pred_dhw, lab_dhw)
        dice_bg, dice_edema, dice_netc, dice_et = dices
        dice_wt = dice_whole_tumor(pred_dhw, lab_dhw)
        mean_tumor = float(np.mean(dices[1:]))

        print(f"  Dice: ED={dice_edema:.3f}  NETC={dice_netc:.3f}  ET={dice_et:.3f}  WT={dice_wt:.3f}  mean(1–3)={mean_tumor:.3f}")

        out_nii_path = os.path.join(OUTPUT_ROOT, f"{case_id}_UNetFLAIR_pred.nii.gz")
        save_pred_as_nifti(pred_dhw, img_path, out_nii_path)

        all_results.append({
            "case_id": case_id,
            "image_path": img_path,
            "label_path": lab_path,
            "pred_mask_path": out_nii_path,
            "dice_bg": float(dice_bg),
            "dice_edema": float(dice_edema),
            "dice_netc": float(dice_netc),
            "dice_et": float(dice_et),
            "dice_whole_tumor": float(dice_wt),
            "dice_mean_tumor_classes": float(mean_tumor),
        })

    if all_results:
        import pandas as pd
        df = pd.DataFrame(all_results)
        csv_path = os.path.join(OUTPUT_ROOT, "UNetFLAIR_MSD_test_results.csv")
        df.to_csv(csv_path, index=False)
        print("\nSaved CSV:", csv_path)

    print("\n✅ Done U-Net FLAIR-only MSD test inference.")

if __name__ == "__main__":
    run_inference()


Using device: cuda
Loaded test split: 97 cases


U-Net FLAIR-only MSD test inference:   0%|          | 0/97 [00:00<?, ?it/s]


Case: BRATS_137
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_137.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_137.nii.gz
  Dice: ED=0.720  NETC=0.338  ET=0.409  WT=0.913  mean(1–3)=0.489


U-Net FLAIR-only MSD test inference:   1%|          | 1/97 [00:02<04:28,  2.80s/it]


Case: BRATS_441
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_441.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_441.nii.gz


...

  Dice: ED=0.839  NETC=0.449  ET=0.677  WT=0.918  mean(1–3)=0.655

Case: BRATS_028
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_028.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_028.nii.gz


U-Net FLAIR-only MSD test inference:  99%|█████████▉| 96/97 [02:41<00:01,  1.67s/it]

  Dice: ED=0.393  NETC=0.173  ET=0.132  WT=0.870  mean(1–3)=0.233

Case: BRATS_455
 Image: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/imagesTr/BRATS_455.nii.gz
 Label: /content/drive/MyDrive/Task01_BrainTumour_extracted/Task01_BrainTumour/labelsTr/BRATS_455.nii.gz


U-Net FLAIR-only MSD test inference: 100%|██████████| 97/97 [02:42<00:00,  1.68s/it]

  Dice: ED=0.862  NETC=0.397  ET=0.561  WT=0.953  mean(1–3)=0.606

Saved CSV: /content/drive/MyDrive/MSD_UNetFLAIR_test_preds/UNetFLAIR_MSD_test_results.csv

✅ Done U-Net FLAIR-only MSD test inference.
